In [135]:
import pandas as pd
import numpy as np
import missingno as miss


In [136]:
## import csvs
df1 = pd.read_csv("county_ages.csv") #age demographics
df2 = pd.read_csv("county_demographics.csv") #countydemographics
df3 = pd.read_csv("countypres_2000-2016.csv") #2016 voting results
df4 = pd.read_csv("2020_county_results.csv") #2020 voting results


In [137]:
## only select results from 2016
df3 = df3[df3['year'] == 2016].groupby('FIPS').mean().reset_index()

In [138]:
## fix fips format for those that begin with zero

def fips_format(df, column):
    df['fips'] = df[column].fillna(0.0).astype(int)
    df['fips'] = df['fips'].astype(str)
    df['fips'] = df['fips'].str.zfill(5)

In [139]:
fips_format(df1, 'fips')
fips_format(df2, 'fips')
fips_format(df3, 'FIPS')
fips_format(df4, 'fips')


In [140]:
## change age columns to match what I want
df1 = df1.drop(['Unnamed: 0'], axis = 1)
df2 = df2.drop(['Unnamed: 0'], axis = 1)

## MALE
## less than 30
m_le_30 = ['M18-19', 'M20', 'M21', 'M22-24', 'M25-29']
df1['male_le_30'] = df1[m_le_30].sum(axis = 1)
df1 = df1.drop(df1[m_le_30], axis = 1)

## 30 - 45
m_ge_30_le_45 = ['M30-34', 'M35-39', 'M40-44']
df1['male_ge_30_le_45'] = df1[m_ge_30_le_45].sum(axis = 1)
df1 = df1.drop(df1[m_ge_30_le_45], axis = 1)

## 45 - 65
m_ge_45_le_65 = ['M45-49', 'M50-54', 'M55-59', 'M60-61', 'M62-64']
df1['male_ge_45_le_65'] = df1[m_ge_45_le_65].sum(axis = 1)
df1 = df1.drop(df1[m_ge_45_le_65], axis = 1)

## 65+
m_ge_65 = ['M65-66', 'M67-69', 'M70-74', 'M76-79', 'M80-84', 'M85+']
df1['male_ge_65'] = df1[m_ge_65].sum(axis = 1)
df1 = df1.drop(df1[m_ge_65], axis = 1)


## FEMALE
## less than 30
f_le_30 = ['W18-19', 'W20', 'W21', 'W22-24', 'W25-29']
df1['female_le_30'] = df1[f_le_30].sum(axis = 1)
df1 = df1.drop(df1[f_le_30], axis = 1)

## 30 - 45
f_ge_30_le_45 = ['W30-34', 'W35-39', 'W40-44']
df1['female_ge_30_le_45'] = df1[f_ge_30_le_45].sum(axis = 1)
df1 = df1.drop(df1[f_ge_30_le_45], axis = 1)

## 45 - 65
f_ge_45_le_65 = ['W45-49', 'W50-54', 'W55-59', 'W60-61', 'W62-64']
df1['female_ge_45_le_65'] = df1[f_ge_45_le_65].sum(axis = 1)
df1 = df1.drop(df1[f_ge_45_le_65], axis = 1)

## 65+
f_ge_65 = ['W65-66', 'W67-69', 'W70-74', 'W76-79', 'W80-84', 'W85+']
df1['female_ge_65'] = df1[f_ge_65].sum(axis = 1)
df1 = df1.drop(df1[f_ge_65], axis = 1)

## join male and female
df1['age_le_30'] = df1['female_le_30'] + df1['male_le_30']
df1['age_ge_30_le_45'] = df1['female_ge_30_le_45'] + df1['male_ge_30_le_45']
df1['age_ge_45_le_65'] = df1['female_ge_45_le_65'] + df1['male_ge_45_le_65']
df1['age_ge_65'] = df1['female_ge_65'] + df1['male_ge_65']


In [141]:
df1.head()

,NAME,M5,M5-9,M10-14,M15-17,W5,W5-9,W10-14,W15-17,state,...,male_ge_45_le_65,male_ge_65,female_le_30,female_ge_30_le_45,female_ge_45_le_65,female_ge_65,age_le_30,age_ge_30_le_45,age_ge_45_le_65,age_ge_65
0,"Fayette County, Illinois",11469,621,621,417,564,497,711,387,17,...,3203,1864,1327,1700,2716,2194,3213,3921,5919,4058
1,"Logan County, Illinois",14298,828,757,544,725,718,784,503,17,...,3868,2248,2588,2673,3738,2976,5005,5561,7606,5224
2,"Saline County, Illinois",11807,798,612,546,700,571,774,392,17,...,3215,2103,1588,2076,3404,2682,3260,4134,6619,4785
3,"Lake County, Illinois",350466,20725,24406,16798,19978,22550,25428,15928,17,...,95795,43532,50870,63929,99517,52807,110995,127281,195312,96339
4,"Massac County, Illinois",6714,388,516,247,403,712,325,269,17,...,1885,1268,860,1271,1971,1694,1759,2438,3856,2962


In [142]:
## merge dataframes
df = pd.merge(df1,df2, how='left', on=['NAME','state', 'county', 'fips'])
df['fips'] = df['fips'].astype(str)
df4['fips'] = df4['fips'].astype(str)
df = pd.merge(df,df3[['fips', 'totalvotes']], how='left', left_on=['fips'], right_on=['fips'])
df = pd.merge(df,df4[['fips','votes']], how='left', left_on=['fips'], right_on=['fips'])



In [143]:
df = df.rename(columns={"totalvotes": "total_votes_2016", "votes": "total_votes_2020"})

In [144]:
## export as csv
df.to_csv('county_data.csv')

In [145]:
df['age_le_30']

0         3213
1         5005
2         3260
3       110995
4         1759
         ...  
3215      1962
3216      1392
3217     87553
3218     29926
3219     69792
Name: age_le_30, Length: 3220, dtype: int64